Код с документации HuggingFace

In [46]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
inputs = tokenizer('@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@', return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=10,
    top_p=0.95,
    num_beams=3,
    num_return_sequences=3,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.2,
    repetition_penalty=1.2,
    length_penalty=1.0,
    eos_token_id=50257,
    max_new_tokens=40
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response


loading file vocab.json from cache at C:\Users\Daniel/.cache\huggingface\hub\models--tinkoff-ai--ruDialoGPT-medium\snapshots\e51fe3a6ea7037f3f53938e3a58ee6e40a82fce3\vocab.json
loading file merges.txt from cache at C:\Users\Daniel/.cache\huggingface\hub\models--tinkoff-ai--ruDialoGPT-medium\snapshots\e51fe3a6ea7037f3f53938e3a58ee6e40a82fce3\merges.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at C:\Users\Daniel/.cache\huggingface\hub\models--tinkoff-ai--ruDialoGPT-medium\snapshots\e51fe3a6ea7037f3f53938e3a58ee6e40a82fce3\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\Daniel/.cache\huggingface\hub\models--tinkoff-ai--ruDialoGPT-medium\snapshots\e51fe3a6ea7037f3f53938e3a58ee6e40a82fce3\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\Daniel/.cache\huggingface\hub\models--tinkoff-ai--ruDialoGPT-medium\snapshots\e51fe3a6ea7037f3f53938e3a58ee6e40a82fce3\tokenizer_config.json
Adding

['@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@нормально, а у тебя? Как сам? Что нового в жизни? Я вот на работе сижу и думаю, чем заняться. А ты? Чем занимаешься? =) ^_^@@ВТОРОЙ@@',
 '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@привет  хорошо. а у тебя как? что делаешь? чем занимаешься? я тоже хочу погулять с тобой 👉🏻\u200d♀️�',
 '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@нормально, а у тебя? Как сама? Чем занимаешься? Что делаешь? Я вот сижу в интернете и не понимаю, что происходит. 🤷\u200d♂�']

В качестве открытого чата возьмём CCM - чат Chess Club Moscow, в котором обсуждают шахматы и договариваются об играх и турнирах в клубе: https://t.me/pairwccm

In [47]:
import json
with open("ccm.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [48]:
data["messages"][0].keys()

dict_keys(['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'title', 'text', 'text_entities'])

In [49]:
#data

Удаляем пустые сообщения и прочие артефакты. Сохраняем идентификаторы отправителей

In [50]:
condition = lambda msg: len(msg["text"]) > 0 and type(msg["text"]) == str

raw_texts = [msg["text"] for msg in data["messages"] if condition(msg)]
users = [msg.get("from_id") for msg in data["messages"] if condition(msg)]

При обработке сообщений если один и тот же пользователь отправляет несколько сообщений подряд, то склеиваем их в одно сообщение. Так как при обучении исходной модели использовались метки @@ПЕРВЫЙ@@ и @@ВТОРОЙ@@, то добавим их в нашу выборку: если прошлое сообщение отправил @@ПЕРВЫЙ@@, то следующее отправляет @@ВТОРОЙ@@ и наоборот. Эти метки добавляем перед сообщением.

Также я пробовал другой подход: чтобы модель лучше понимала, как правильно отвечать на вопросы, я пробовал добавлять каждое новое сообщение к предыдущему, то есть "@@ПЕРВЫЙ@@ <сообщение 1> @@ВТОРОЙ@@ <сообщение 2>; @@ВТОРОЙ@@ <сообщение 2> @@ПЕРВЫЙ@@ <сообщение 3>...", но при таком подходе качество сгенерированных сообщений сильно ухудшилось

In [53]:
first = "@@ПЕРВЫЙ@@ "
second = "@@ВТОРОЙ@@ "
texts = [first + raw_texts[0]]
last = 0
for i in range(1, len(raw_texts)):
    if users[i] == users[i-1]:
        texts[-1] += ". " + raw_texts[i]
    else:
        #texts[-1] += second + raw_texts[i]
        #texts.append(first + raw_texts[i])
        last = (last + 1) % 2
        if not last:
            texts.append(first + raw_texts[i])
        else:
            texts.append(second + raw_texts[i])

In [14]:
"""
first = "@@ПЕРВЫЙ@@ "
second = " @@ВТОРОЙ@@ "
texts = [first + raw_texts[0]]
#last = 0
for i in range(1, len(raw_texts)):
    if users[i] == users[i-1]:
        texts[-1] += ". " + raw_texts[i]
    else:
        texts[-1] += second + raw_texts[i]
        texts.append(first + raw_texts[i])
        #last = (last + 1) % 2
        #if not last:
        #    texts.append(first + raw_texts[i])
        #else:
        #    texts.append(second + raw_texts[i])
"""

In [55]:
texts[:10]

['@@ПЕРВЫЙ@@ Здравствуйте всем. Я хоть и плохо играю в шахматы) но охото поиграть с кем нибудь в новом году',
 '@@ВТОРОЙ@@ Всем привет! Присоединяюсь 😁',
 '@@ПЕРВЫЙ@@ Заработало))) рад всех видеть))',
 '@@ВТОРОЙ@@ Эх жалко прям перед каникулами клуба)',
 '@@ПЕРВЫЙ@@ Это задел на светлое будущее)',
 '@@ВТОРОЙ@@ Чтобы никто не остался без партнёра по игре в 2020! 😁',
 '@@ПЕРВЫЙ@@ Добавляйте своих шахматных друзей, к сожалению у нас есть не все телефоны...',
 '@@ВТОРОЙ@@ Привет! И с Наступающим! \nНадо собраться в формате 20/20 в новом году!',
 '@@ПЕРВЫЙ@@ Очень круто , что группа создана, пару раз приезжал в WCC и пусто) теперь думаю все будет ок!)))',
 '@@ВТОРОЙ@@ С Наступающим НГ! офигенный бар!']

In [56]:
from transformers import Trainer, TrainingArguments

Токенизируем собранные тексты

In [57]:
inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=256)
inputs

{'input_ids': tensor([[50257, 10288,  1514,  ...,     0,     0,     0],
        [50258,  7914,  6129,  ...,     0,     0,     0],
        [50257,   999,  2050,  ...,     0,     0,     0],
        ...,
        [50258,  9543,    16,  ...,     0,     0,     0],
        [50257,  5452,  2696,  ...,     0,     0,     0],
        [50258,   981,    16,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Создаем датасет и разделяем его на обучающий и валидационный в пропорции 4:1

In [58]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.data["labels"] = data["input_ids"]

    def __len__(self):
        return len(self.data["input_ids"])

    def __getitem__(self, idx):
        return {key: value[idx] for key, value in self.data.items()}

threshold = int(0.8 * len(inputs.data["input_ids"]))
train_data = {
    key: value[:threshold] for key, value in inputs.data.items()
}
val_data = {
    key: value[threshold:] for key, value in inputs.data.items()
}

train_dataset = TextDataset(train_data)
val_dataset = TextDataset(val_data)

В качестве максимальной длины токеннизированного сообщения я выбрал 256. Посмотрим, в скольких сообщениях нет паддинга (возможно, они обрезаны)

In [73]:
cnt = 0
for x in train_dataset:
    if x["attention_mask"][-1] == 1:
        cnt += 1
        #rint(x)
cnt

31

In [60]:
print(len(train_dataset))

33612


Таких сообщений получилось очень мало, и вряд ли обрезанная часть несет какую-то важную информацию, поэтому я считаю 256 оптимальным вариантом

У моделей HuggingFace есть встроенные методы для дообучения, используя которые не нужно самостоятельно создавать классы моделей и настраивать пайплайн обучения.

Сохраним исходную модель, зададим параметры обучения и запустим его.

In [62]:
model.save_pretrained("original_model")

Configuration saved in original_model\config.json
Model weights saved in original_model\pytorch_model.bin


In [61]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    num_train_epochs=1, #при обучении на нескольких эпохах модель заметно переобучается
    save_steps=5000,
    output_dir='./model_output',
    overwrite_output_dir=True,
    do_train=True,
    disable_tqdm = False,
    evaluation_strategy="steps",
    eval_steps=5000
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Проверим качество оригинальной модели на валидационных данных до начала дообучения

In [65]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 8404
  Batch size = 8


{'eval_loss': 27.32219886779785,
 'eval_runtime': 828.4204,
 'eval_samples_per_second': 10.145,
 'eval_steps_per_second': 1.269}

In [66]:
trainer.train()

C:\Users\Daniel\anaconda3\envs\dl\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33612
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 16806
  Number of trainable parameters = 355875840


Step,Training Loss,Validation Loss
5000,0.273400,0.356852
10000,0.305400,0.350833
15000,0.275700,0.345955


***** Running Evaluation *****
  Num examples = 8404
  Batch size = 8
Saving model checkpoint to ./model_output\checkpoint-5000
Configuration saved in ./model_output\checkpoint-5000\config.json
Model weights saved in ./model_output\checkpoint-5000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8404
  Batch size = 8
Saving model checkpoint to ./model_output\checkpoint-10000
Configuration saved in ./model_output\checkpoint-10000\config.json
Model weights saved in ./model_output\checkpoint-10000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8404
  Batch size = 8
Saving model checkpoint to ./model_output\checkpoint-15000
Configuration saved in ./model_output\checkpoint-15000\config.json
Model weights saved in ./model_output\checkpoint-15000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=16806, training_loss=0.2978854185624391, metrics={'train_runtime': 20351.398, 'train_samples_per_second': 1.652, 'train_steps_per_second': 0.826, 'total_flos': 1.5607743872237568e+16, 'train_loss': 0.2978854185624391, 'epoch': 1.0})

После дообучения сохраним модель

In [68]:
model.save_pretrained("fine-tuned ccm2")

Configuration saved in fine-tuned ccm2\config.json
Model weights saved in fine-tuned ccm2\pytorch_model.bin


Несмотря на то, что при дообучении была передана маска внимания, символы паддинга все равно попадают в генерацию. Очистим сгенерированный текст от них, а также от меток @@ПЕРВЫЙ@@ и @@ВТОРОЙ@@

In [74]:
def extract_response(text, start_marker="@@ВТОРОЙ@@", end_marker="@@ПЕРВЫЙ@@"):
    text = re.sub(r'<pad>', '', text)
    
    match = re.search(f"{re.escape(start_marker)}(.+?){re.escape(end_marker)}", text)
    if match:
        return match.group(1).strip()
    
    match = re.search(f"{re.escape(start_marker)}(.+)", text)
    if match:
        return match.group(1).strip()
    
    return "ОШИБКА"

In [130]:
model.to('cpu')
inputs = tokenizer('@@ПЕРВЫЙ@@ привет, ты играешь в шахматы? @@ВТОРОЙ@@ ', return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=10,
    top_p=0.95,
    num_beams=5,
    num_return_sequences=3,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.3,
    repetition_penalty=1.2,
    length_penalty=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=40
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response_cleaned = [extract_response(text) for text in context_with_response]
context_with_response_cleaned

Setting `pad_token_id` to `eos_token_id`:50261 for open-end generation.


['🤣🙌🏻\u200d♀️',
 '🤣👌🏻🙏💫. я бы с удовольствием поиграл, но у меня нет компа, поэтому и не знаю, как это сделать)',
 '🤣🙈. а какой у тебя рейтинг?']

Норм

Теперь обернем это в бота: https://t.me/chess_club_moscow_gpt_bot